In [1]:
%defaultDatasource jdbc:h2:mem:db

# Zombie Survey Dataset (`zombies-survey.csv`)
* name - Zombie name
* type - biological, chemical or technological
* gender - the gender before becoming a zombie
* weight - weight in kg
* height - height in m
* GBC - Green Blood Cells (million cells per microliter - mcL)
* GS - Green Substance (g/dL - grams per deciliter)
* PBC - Purple Blood Cells (million cells per microliter - mcL)
* SP - Sticky Paste (g/dL - grams per deciliter)

In [2]:
DROP TABLE IF EXISTS Survey;

CREATE TABLE Survey (
  name VARCHAR(50),
  type VARCHAR(13) NOT NULL,
  gender VARCHAR(6) NOT NULL,
  height DECIMAL(4,1),
  weight DECIMAL(6,2),
  gbc DECIMAL(4,1),
  gs INTEGER,
  pbc DECIMAL(4,1),
  sp DECIMAL(4,1),
  stature VARCHAR(7),
  PRIMARY KEY(name)
) AS SELECT
  name,type,gender,height,weight,gbc,gs,pbc,sp,'regular'
FROM CSVREAD('../../data/zombie/zombie-health/zombies-survey.csv');

In [3]:
SELECT * FROM Survey;

## Caracterização (Feature)

### Estatura: alta, média e baixa

In [4]:
UPDATE Survey
SET stature = 'short' WHERE height < 1.6;

UPDATE Survey
SET stature = 'tall' WHERE height > 1.85;

In [5]:
SELECT * FROM Survey;

## Pergunta: *Há alteração sigficativa da média do peso de acordo com a estatura?*

In [6]:
SELECT stature, AVG(weight) FROM Survey GROUP BY stature ORDER BY AVG(weight);

## Pergunta: *O tipo de zumbi afeta na distribuição das alturas?*

In [7]:
SELECT type, stature, COUNT(*) FROM Survey GROUP BY type, stature ORDER BY type, stature;

## Pergunta: *Há diferença de altura entre zumbis masculinos e femininos?*

In [8]:
SELECT gender, stature, COUNT(*) FROM Survey GROUP BY gender, stature ORDER BY gender, stature;

# National Health and Nutrition Examination Survey
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

## NHANES datasets from 2013-2014

This dataset was extracted from [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013). The following tables are available:

### `demographic-person.csv`

Persons who participated in the survey.

| field | description | domain |
|-------|-------------|--------|
| `id` | id of a participant | number |
| `gender` | gender of the participant | `1` - male; `2` - female |
| `age` | age of the participant | number |

### `medications-drug.csv`

Drugs catalog.

| field | description | domain |
|-------|-------------|--------|
| `code` | code of the drug | string |
| `name` | name of the drug | string |

### `medications-use.csv`

Records drugs used by participants in the survey and how many days they are continuously using the drug.

| field | description | domain |
|-------|-------------|--------|
| `person_id` | id of a participant in the survey | number |
| `drug_code` | code of a drug in the catalog | string |
| `days_use` | number of days of use |

In [9]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Medicamento;
DROP TABLE IF EXISTS UsoMedicamento;

CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  FaixaEtaria VARCHAR(11),
  NumeroMedicamentos FLOAT,
  MedPoucos BOOLEAN,
  MedMuitos BOOLEAN,
  MedAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age,
    'adulto',
    0,
    NULL,
    NULL,
    FALSE
FROM CSVREAD('../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    code,
    name
FROM CSVREAD('../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../data/nhanes/medications-use.csv');

In [10]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

## Caracterizações (features) para Análise de comportamento estatístico
* **NumeroMedicamentos** - número de medicamentos tomados por pessoa
* **MedPoucos** - pessoas que tomam poucos medicamentos (até um medicamento)
* **MedMuitos** - pessoas que tomam muitos medicamentos (três ou mais medicamentos)
* **MedAcimaMedia** - pessoas que tomam mais medicamentos do que a média

In [11]:
Update Pessoa p
SET NumeroMedicamentos = (SELECT COUNT(CodMedicamento) NumeroMedicamentos
                          FROM UsoMedicamento um
                          WHERE um.IdPessoa = p.Id);

In [12]:
SELECT * FROM Pessoa;

In [13]:
SELECT AVG(NumeroMedicamentos) FROM Pessoa WHERE NumeroMedicamentos > 0;

3.4753285395487232

In [14]:
Update Pessoa
SET FaixaEtaria = 'crianca'
WHERE Idade <= 10;

Update Pessoa
SET FaixaEtaria = 'adolescente'
WHERE Idade > 10 AND Idade <= 17;

Update Pessoa
SET FaixaEtaria = 'meia-idade'
WHERE Idade >= 40 AND Idade <= 64;

Update Pessoa
SET FaixaEtaria = 'idoso'
WHERE Idade > 64;

In [15]:
Update Pessoa
SET MedPoucos = TRUE
WHERE NumeroMedicamentos <= 1;

Update Pessoa
SET MedMuitos = TRUE
WHERE NumeroMedicamentos >= 3;

Update Pessoa
SET MedAcimaMedia = TRUE
WHERE NumeroMedicamentos > (SELECT AVG(NumeroMedicamentos) FROM Pessoa WHERE NumeroMedicamentos > 0);

In [16]:
SELECT * FROM Pessoa;

## Pergunta: *Qual a distribuição de uso de número de medicamentos por faixa etária?*

In [17]:
SELECT FaixaEtaria, AVG(NumeroMedicamentos) Media
FROM Pessoa WHERE NumeroMedicamentos > 0
GROUP BY FaixaEtaria ORDER BY FaixaEtaria;

## Discriminação de Dados

### Muitos Medicamentos x Poucos Medicamentos

* 96.5% das pessoas que tomam poucos medicamentos não é idoso
* 85.7% das pessoas que tomam muitos medicamentos é de meia idade ou idoso.

In [18]:
SELECT COUNT(*) FROM Pessoa WHERE MedPoucos = TRUE AND Idade >= 18;

3636

In [19]:
SELECT COUNT(*) FROM Pessoa WHERE MedMuitos = TRUE;

2013

In [20]:
SELECT
(SELECT CAST(COUNT(*) AS FLOAT) FROM Pessoa WHERE MedPoucos = TRUE AND (FaixaEtaria <> 'idoso')) /
(SELECT COUNT(*) FROM Pessoa WHERE MedPoucos = TRUE) * 100;

96.48484848484848

In [21]:
SELECT
(SELECT CAST(COUNT(*) AS FLOAT) FROM Pessoa WHERE MedMuitos = TRUE AND
                                                  (FaixaEtaria = 'meia-idade' OR FaixaEtaria = 'idoso')) /
(SELECT COUNT(*) FROM Pessoa WHERE MedMuitos = TRUE) * 100;

85.6929955290611

## Pergunta: *Qual a proporção de pessoas que tomam poucos medicamentos versus que tomam muitos medicamentos por faixa etária?*

In [22]:
SELECT FaixaEtaria, COUNT(MedPoucos) Poucos, COUNT(MedMuitos) Muitos,
       Cast(COUNT(MedPoucos) AS FLOAT) / COUNT(MedMuitos) Proportion
FROM Pessoa GROUP BY FaixaEtaria;

## Pergunta: *Há diferença significativa de média de uso de medicamentos de acordo com o gênero, dentro das faixas etárias?*

In [23]:
SELECT FaixaEtaria, Genero, AVG(NumeroMedicamentos) Media
FROM Pessoa WHERE NumeroMedicamentos > 0
GROUP BY FaixaEtaria, Genero ORDER BY FaixaEtaria, Genero;

In [24]:
DROP VIEW FaixaIdade IF EXISTS;

CREATE VIEW FaixaIdadeMed AS
SELECT NumeroMedicamentos, MIN(Idade) Minima, MAX(Idade) Maxima
FROM Pessoa
GROUP BY NumeroMedicamentos ORDER BY NumeroMedicamentos;

In [25]:
SELECT * FROM FaixaIdadeMed;

In [26]:
SELECT AVG(Minima) MediaMinima, AVG(Maxima) MediaMaxima
FROM FaixaIdadeMed
WHERE NumeroMedicamentos <= 10;

In [27]:
SELECT AVG(Minima) MediaMinima, AVG(Maxima) MediaMaxima
FROM FaixaIdadeMed
WHERE NumeroMedicamentos > 10;

## Exportando tabela em CSV para Análise no Orange

In [28]:
UPDATE Pessoa
SET MedPoucos = FALSE
WHERE MedPoucos IS NULL;

UPDATE Pessoa
SET MedMuitos = FALSE
WHERE MedMuitos IS NULL;

In [29]:
CALL CSVWRITE('../../data/nhanes/demographic-person-enriched.csv', 'SELECT * FROM Pessoa');

10175